## Fugue API

[![Slack Status](https://img.shields.io/badge/slack-join_chat-white.svg?logo=slack&style=social)](http://slack.fugue.ai)

All of our discussion by this point has been about the `transform()` function. While the `transform()` function can already handle a lot, it's not enough to describe end-to-end framework-agnostic workflows.

Take a look at the previous examples. We're always creating a DataFrame with Pandas before the distributed operation. If we load with Spark, the workflow becomes dependent on Spark.

```
df = ...
sdf = transform(..., engine="spark")
sdf.write.parquet(...)
```

This is where the broader Fugue API comes in. Fugue has a suite of standalone functions that are all compatible with Pandas, Spark, Dask, and Ray DataFrames. First, we take a look at loading and saving.

## Saving and Loading

Let's setup a DataFrame and save it for use in this section.

In [1]:
import pandas as pd

df = pd.DataFrame({"col1": ["a", "a", "a", "b", "b", "b"],
                   "col2": [1, 4, 2, 5, 3, 2]})
df.to_parquet("/tmp/test.parquet")

Now we can see the engine agnostic functions.

In [4]:
import fugue.api as fa

def add_new_col(df: pd.DataFrame) -> pd.DataFrame:
    return df.assign(col3 = df['col2'] * 3)

# Using Pandas
df = fa.load("/tmp/test.parquet")
tdf = fa.transform(df, add_new_col, schema="*, col3: int")
fa.save(tdf, "/tmp/output.parquet")

To check the results, we can use Pandas again:

In [5]:
pd.read_parquet("/tmp/output.parquet")

,col1,col2,col3
0,a,1,3
1,a,4,12
2,a,2,6
3,b,5,15
4,b,3,9
5,b,2,6


## Bringing to Spark